In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hf-model/config.json
/kaggle/input/hf-model/merges.txt
/kaggle/input/hf-model/tokenizer.json
/kaggle/input/hf-model/vocab.json
/kaggle/input/hf-model/tokenizer_config.json
/kaggle/input/hf-model/pytorch_model.bin
/kaggle/input/hf-model/special_tokens_map.json
/kaggle/input/model-2/config.json
/kaggle/input/model-2/pytorch_model.bin
/kaggle/input/detection/Untitled.ipynb
/kaggle/input/detection/modeltorch/config.json
/kaggle/input/detection/modeltorch/model.safetensors
/kaggle/input/detection/saved models/tokenizer_config.json
/kaggle/input/detection/saved models/special_tokens_map.json
/kaggle/input/detection/saved models/vocab.txt
/kaggle/input/detection/saved models/added_tokens.json
/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv
/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv
/kaggle/input/tokenizer/tokenizer/tokenizer.json
/kaggle/input/tokenizer/tokenizer/tokenizer_config.json
/kaggle/input/tokenizer/tokenizer/special_tokens_m

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AdamW

from torch.utils.data import DataLoader
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class gptDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = [0 for _ in encodings['input_ids']] if labels==None else labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [4]:
saved_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/hf-model')

In [5]:
saved_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/hf-model')

In [6]:
train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
train

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [7]:
train_texts = train['text'].tolist()

In [8]:
test_encodings = saved_tokenizer(train_texts, truncation=True, padding=True)

In [9]:
dataset = gptDataset(test_encodings)

In [10]:
loader = DataLoader(dataset)

In [11]:
# gpt3 = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv')
# gpt4 = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv')
# gpt = pd.concat([train,gpt3,gpt4])
# gen = gpt['text'].to_list()
# tags = gpt['generated'].to_list()
# gen = saved_tokenizer(gen, truncation=True, padding=True)
# dataset1 = gptDataset(gen,tags)

In [12]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print(device)
# model = DistilBertForSequenceClassification.from_pretrained('/kaggle/input/model-2')
# model.to(device)
# model.train()

# train_loader = DataLoader(dataset1, batch_size=8, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(30):
#     print(f"Epoch : {epoch+1}--")
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()

In [13]:
# model.save_pretrained('/kaggle/working')

In [14]:
saved_model.to('cpu')

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [15]:
preds=[]
saved_model.eval()
i=1
for batch in loader:
#     print(i)
    i+=1
    batch = {k: v for k, v in batch.items()}
    with torch.no_grad():
        outputs = saved_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    preds.append(predictions[0].item())

In [16]:
preds

[0, 0, 0]

In [17]:
train_id = train['id'].tolist()

In [18]:
submission = pd.DataFrame({'id':train_id,'generated':preds})

In [19]:
submission

,id,generated
0,0000aaaa,0
1,1111bbbb,0
2,2222cccc,0


In [20]:
submission.to_csv('/kaggle/working/submission.csv',index=False)

In [21]:
# from IPython.display import FileLink 
# FileLink('/kaggle/working/model/config.json')

In [22]:
# import os
# import subprocess
# from IPython.display import FileLink, display

# def download_file(path, download_file_name):
#     os.chdir('/kaggle/working/')
#     zip_name = f"/kaggle/working/{download_file_name}.zip"
#     command = f"zip {zip_name} {path} -r"
#     result = subprocess.run(command, shell=True, capture_output=True, text=True)
#     if result.returncode != 0:
#         print("Unable to run zip command!")
#         print(result.stderr)
#         return
#     display(FileLink(f'{download_file_name}.zip'))

In [23]:
# download_file('/kaggle/working/','out')